In [3]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [115]:
def get_Score(url):
  try:
      page=requests.get(url)                             # this might throw an exception if something goes wrong
  except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)
  time.sleep(2)   
  soup=BeautifulSoup(page.text,'html.parser')
  #links=soup.find_all('article',attrs={'class':'o-platform__content'})
  links=soup.find_all('div',attrs={'class':'m-scorecard__body'})
  #print(len(links))
  Score = []

  for j in links:
    TrueScore = j.find('p').find('a').text.strip()
    Score.append(TrueScore)
  return Score

In [ ]:
s = get_Score('https://www.politifact.com/personalities/facebook-posts/')
print(s[5])

In [ ]:
pagesToGet= 
upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)

    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    filename="NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="label, statement, speaker, true, mostly_true, half_true, mostly_false, false, pants_on_fire, source, link ,date\n"
    f.write(headers)

    for j in links:
      label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
      statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
      speaker = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
      Personality =  "https://www.politifact.com" + j.find('div',attrs={'class':'m-statement__meta'}).find('a')['href'].strip()
      Score = get_Score(Personality)
      true_score = Score[0]
      mostly_true = Score[1]
      half_true = Score[2]
      mostly_false = Score[3]
      false_score = Score[4]
      pants_on_fire = Score[5]
      source = j.find("div",attrs={'class':'m-statement__desc'}).text.strip()
      link = "https://www.politifact.com"
      link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
      date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
      frame.append((label, statement, speaker, true_score, mostly_true, half_true, mostly_false, false_score, pants_on_fire, source, link ,date))
      #frame.append((Statement,Venue,Link,Date,Source,Label))
      #f.write(Statement.replace(",","^")+","+Venue.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
#f.close()
data=pd.DataFrame(upperframe, columns=['label', 'statement', 'speaker', 
                                       'true_score', 'mostly_true', 'half_true', 
                                       'mostly_false', 'false_score', 'pants_on_fire', 'source', 
                                       'link' ,'date'])
data.head(10)

In [119]:
len(data)

90

In [120]:
data.to_csv("polifact-2.csv")

In [2]:
# from bs4 import BeautifulSoup
# import pandas as pd
# from tqdm import tqdm
# from os import path
# import requests
# import hashlib
# import time

# TOTAL_PAGES = 573
# col_index = ['name', 'quote_desc', 'quote', 'link', 'date_line', 'rating', 'sha256']
# local_raw_data = "./politifact_data.csv"


# def get_politifacts_page(page_num):
#     page = requests.get(f"https://www.politifact.com/factchecks/list/?page={page_num}")
#     soup = BeautifulSoup(page.content, 'html.parser')
#     page_data = pd.DataFrame(columns=col_index)
#     items = soup.findAll('article', class_="m-statement")
#     for item in items:
#         # Who (or what) is the quote attributed to
#         name = item.find('a', class_="m-statement__name").get_text().strip()
#         # Where was the quote made?
#         quote_desc = item.find('div', class_="m-statement__desc").get_text().strip()
#         # What is the quote
#         quote = item.find('div', class_="m-statement__quote").find('a')
#         link = f"https://politifact.com{quote['href'].strip()}"
#         quote_text = quote.get_text().strip()
#         # Date line with attribution, used for SHA-256 signature
#         date_line = item.find('footer', class_="m-statement__footer").get_text().strip()
#         # Rating - Label to be predicted. Might filter some of these out later
#         rating = item.find('div', class_="m-statement__meter").find('picture').find('img')['alt']

#         # SHA 256 hash used for identifying duplicate entries.
#         # Will also be converted to an int for repeatable train/validation/test splits.
#         sha256 = hashlib.sha256(f"{name}{quote_desc}{quote_text}{link}{date_line}{rating}".encode()).hexdigest()

#         new_row = {col_index[0]: name, col_index[1]: quote_desc, col_index[2]: quote_text, col_index[3]: link,
#                    col_index[4]: date_line, col_index[5]: rating, col_index[6]: sha256}
#         page_data = page_data.append(new_row, ignore_index=True)
#     return page_data


# def update_politifact_data_set(end_page=1, start_page=1, data_set=None, wait_time=3):
#     if data_set is None:
#         data_set = pd.DataFrame(columns=col_index)

#     if end_page > TOTAL_PAGES:
#         end_page = TOTAL_PAGES

#     for page_number in tqdm(range(end_page, start_page - 1, -1)):
#         data_set = get_politifacts_page(page_num=page_number).append(data_set, ignore_index=True)
#         # Delay between page requests. Don't be rude and slam their server.
#         time.sleep(wait_time)

#     # Remove any duplicate entries
#     data_set = data_set.drop_duplicates(subset="sha256")

#     return data_set


# if __name__ == "__main__":
#     if path.exists(local_raw_data):
#         data = pd.read_csv(local_raw_data, sep='|')
#     else:
#         data = None

#     data = update_politifact_data_set(5, data_set=data)

#     data.to_csv(local_raw_data, header=True, index=False, sep='|')

100%|██████████| 5/5 [00:24<00:00,  4.83s/it]
